In [1]:
import torch
from torch import nn
import numpy as np
import matplotlib.pyplot as plt

In [6]:
def corr2d(input_array: torch.Tensor, kernel: torch.Tensor) -> torch.Tensor:
    m, n = input_array.shape
    a, b = kernel.shape
    y = torch.Tensor(m - a + 1, n - b + 1)
    for i in range(y.shape[0]):
        for j in range(y.shape[1]):
            y[i, j] = (input_array[i:i + a, j:j + b] * kernel).sum()
    return y


In [7]:
# Verify the cross-correlation of the matrices:
x = torch.arange(9).reshape(3, 3)
k = torch.arange(4).reshape(2, 2)
corr2d(x, k)

tensor([[19., 25.],
        [37., 43.]])

In [ ]:
class Conv2D(nn.Module):

    def __init__(self, kernel_size: tuple):
        super().__init__()
        self.weight = nn.Parameter(torch.rand(kernel_size))
        self.bias = nn.Parameter(torch.zeros(1))
    
    def forward(self, x):
        return corr2d(x, self.weight) + self.bias

## 卷积核学习

可以通过训练的方法计算一个卷积核，方法如下：

In [8]:
conv2d = nn.Conv2d(in_channels=1, out_channels=1, kernel_size=(1, 2), bias=False)

x = torch.ones(6, 8)
x[:, 2:6] = 0
k = torch.tensor([[1.0, -1.0]])
y = corr2d(x, k)
y

tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])

In [11]:
X = x.reshape((1, 1, 6, 8))
Y = y.reshape((1, 1, 6, 7))
eta = 0.03

In [12]:
for i in range(10):
    y_hat = conv2d(X)
    l: torch.Tensor = (y_hat - Y) ** 2
    conv2d.zero_grad()
    l.sum().backward()
    conv2d.weight.data -= eta * conv2d.weight.grad
    print(f'epoch{i + 1}, loss {l.sum():.3f}')


epoch1, loss 26.337
epoch2, loss 12.887
epoch3, loss 6.623
epoch4, loss 3.573
epoch5, loss 2.014
epoch6, loss 1.177
epoch7, loss 0.708
epoch8, loss 0.434
epoch9, loss 0.270
epoch10, loss 0.170


In [13]:
conv2d.weight.data.reshape((1, 2))

tensor([[ 1.0280, -0.9443]])